In [111]:
import pandas as pd
import yfinance as yf
import numpy as np
from datetime import date
from pandas_datareader.data import DataReader

In [152]:
#historical problems: The vix wasn't created until 1993 and did not include all S&P components until 2002.
#start was selected because this is when the first dates for spy were in yahoo finance
start = date(1993, 2, 8)
#sofr doesn't start until july of 2018 and LIBOR data ends oct 2016. Switch to SOFR
#doesn't really begin until 2020
sofr = DataReader('SOFR30DAYAVG', 'fred', start=start)
cpi = DataReader('CPIAUCSL', 'fred', start=start)
unrate = DataReader('UNRATE', 'fred', start=start)
vix = DataReader('VIXCLS', 'fred', start=start)
spy = DataReader('SPY', 'yahoo', start=start)



In [153]:
vix

,VIXCLS
DATE,
1993-02-08,13.22
1993-02-09,13.48
1993-02-10,13.43
1993-02-11,12.69
1993-02-12,12.38
...,...
2022-06-01,25.69
2022-06-02,24.72
2022-06-03,24.79


In [154]:
unrate

,UNRATE
DATE,
1993-03-01,7.0
1993-04-01,7.1
1993-05-01,7.1
1993-06-01,7.0
1993-07-01,6.9
...,...
2022-01-01,4.0
2022-02-01,3.8
2022-03-01,3.6


In [155]:
libor_rate = pd.read_csv('3-month-libor-rate-historical-chart.csv')
#libor_rate.set_index('DATE')

In [156]:
libor_rate_chopped = libor_rate.iloc[:8715]
libor_rate_chopped 

,DATE,value
0,1986-01-02,8.0000
1,1986-01-03,8.0625
2,1986-01-06,8.1250
3,1986-01-07,8.1250
4,1986-01-08,8.0000
...,...,...
8710,2020-06-24,0.2838
8711,2020-06-25,0.3060
8712,2020-06-26,0.3079
8713,2020-06-29,0.2961


In [157]:
sofr_chopped = sofr.iloc[565:]
sofr_chopped.rename(columns = {'SOFR30DAYAVG': 'value'}, inplace=True)

C:\Users\bng19\AppData\Local\Temp\ipykernel_2284\3835015811.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sofr_chopped.rename(columns = {'SOFR30DAYAVG': 'value'}, inplace=True)


In [158]:
sofr_chopped = sofr_chopped.reset_index()

In [159]:
int_rate = pd.concat([libor_rate_chopped, sofr_chopped])

In [160]:
#note that SOFR takes over from LIBOR in June 30th.  
int_rate['DATE'] = pd.to_datetime(int_rate['DATE'])
int_rate.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9221 entries, 0 to 505
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   DATE    9221 non-null   datetime64[ns]
 1   value   9200 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 216.1 KB


In [161]:
import datetime
import pickle
import warnings

from hmmlearn.hmm import GaussianHMM
from matplotlib import cm, pyplot as plt
from matplotlib.dates import YearLocator, MonthLocator

In [162]:
spy.to_csv('SPY.csv')

In [163]:
vix = vix.reset_index()

In [164]:
spy = spy.reset_index()
spy

,Date,High,Low,Open,Close,Volume,Adj Close
0,1993-02-08,45.125000,44.906250,44.968750,44.968750,596100.0,26.147602
1,1993-02-09,44.812500,44.562500,44.812500,44.656250,122100.0,25.965910
2,1993-02-10,44.750000,44.531250,44.656250,44.718750,379600.0,26.002239
3,1993-02-11,45.125000,44.781250,44.781250,44.937500,19500.0,26.129446
4,1993-02-12,44.875000,44.593750,44.875000,44.593750,42500.0,25.929565
...,...,...,...,...,...,...,...
7383,2022-06-02,417.440002,407.040009,409.420013,417.390015,79609600.0,417.390015
7384,2022-06-03,414.040009,409.510010,412.399994,410.540009,71830800.0,410.540009
7385,2022-06-06,416.609985,410.549988,414.779999,411.790009,57508900.0,411.790009
7386,2022-06-07,416.220001,407.609985,408.100006,415.739990,59053100.0,415.739990


In [165]:
spy.rename({'Date':'DATE'}, inplace=True, axis=1)
spy_edit = spy[['DATE', 'Volume', 'Adj Close']]

In [166]:
bayes_df = spy_edit.merge(vix, on='DATE')
bayes_df 

,DATE,Volume,Adj Close,VIXCLS
0,1993-02-08,596100.0,26.147602,13.22
1,1993-02-09,122100.0,25.965910,13.48
2,1993-02-10,379600.0,26.002239,13.43
3,1993-02-11,19500.0,26.129446,12.69
4,1993-02-12,42500.0,25.929565,12.38
...,...,...,...,...
7382,2022-06-01,86585800.0,409.589996,25.69
7383,2022-06-02,79609600.0,417.390015,24.72
7384,2022-06-03,71830800.0,410.540009,24.79
7385,2022-06-06,57508900.0,411.790009,25.07


In [167]:
bayes_df= bayes_df.merge(int_rate, on='DATE')

In [168]:
bayes_df['Int_Rate'] = bayes_df['value']

In [169]:
bayes_df = bayes_df.drop(['value'], axis=1)

In [170]:
bayes_df = bayes_df.merge(cpi, how='left', on='DATE')

In [171]:
bayes_df = bayes_df.ffill()

In [172]:
bayes_df = bayes_df.bfill()
bayes_df['CPI'] = bayes_df['CPIAUCSL']

In [173]:
bayes_df = bayes_df.merge(unrate, how='left', on='DATE')

In [174]:
bayes_df = bayes_df.ffill()
bayes_df = bayes_df.bfill()

In [175]:
bayes_df.drop(['CPIAUCSL'], axis=1)

,DATE,Volume,Adj Close,VIXCLS,Int_Rate,CPI,UNRATE
0,1993-02-08,596100.0,26.147602,13.22,3.25000,143.300,7.0
1,1993-02-09,122100.0,25.965910,13.48,3.25000,143.300,7.0
2,1993-02-10,379600.0,26.002239,13.43,3.25000,143.300,7.0
3,1993-02-11,19500.0,26.129446,12.69,3.25000,143.300,7.0
4,1993-02-12,42500.0,25.929565,12.38,3.25000,143.300,7.0
...,...,...,...,...,...,...,...
7254,2022-06-01,86585800.0,409.589996,25.69,0.73621,288.663,3.6
7255,2022-06-02,79609600.0,417.390015,24.72,0.75289,288.663,3.6
7256,2022-06-03,71830800.0,410.540009,24.79,0.76923,288.663,3.6
7257,2022-06-06,57508900.0,411.790009,25.07,0.78491,288.663,3.6


In [176]:
bayes_df['SP1'] = bayes_df['Adj Close'].pct_change()
bayes_df['VX1'] = bayes_df['VIXCLS'].pct_change()
bayes_df 

,DATE,Volume,Adj Close,VIXCLS,Int_Rate,CPIAUCSL,CPI,UNRATE,SP1,VX1
0,1993-02-08,596100.0,26.147602,13.22,3.25000,143.300,143.300,7.0,NaN,NaN
1,1993-02-09,122100.0,25.965910,13.48,3.25000,143.300,143.300,7.0,-0.006949,0.019667
2,1993-02-10,379600.0,26.002239,13.43,3.25000,143.300,143.300,7.0,0.001399,-0.003709
3,1993-02-11,19500.0,26.129446,12.69,3.25000,143.300,143.300,7.0,0.004892,-0.055101
4,1993-02-12,42500.0,25.929565,12.38,3.25000,143.300,143.300,7.0,-0.007650,-0.024429
...,...,...,...,...,...,...,...,...,...,...
7254,2022-06-01,86585800.0,409.589996,25.69,0.73621,288.663,288.663,3.6,-0.008089,-0.019091
7255,2022-06-02,79609600.0,417.390015,24.72,0.75289,288.663,288.663,3.6,0.019043,-0.037758
7256,2022-06-03,71830800.0,410.540009,24.79,0.76923,288.663,288.663,3.6,-0.016412,0.002832
7257,2022-06-06,57508900.0,411.790009,25.07,0.78491,288.663,288.663,3.6,0.003045,0.011295


In [180]:
#Create Lead Column 
bayes_df['SP2'] = bayes_df['SP1']
bayes_df['SP2'] = bayes_df.SP2.shift(-1)
bayes_df
bayes_df = bayes_df.iloc[1:]

In [181]:
bayes_df.to_csv('bayes_df.csv')